In [72]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash
# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacUser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard - Tori Adams'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),

    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(
        # RadioItems for filtering animal type
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'View All Animals', 'value': 'All'},
                {'label': 'View All Dogs', 'value': 'Dog'},
                {'label': 'View All Cats', 'value': 'Cat'},
                {'label': 'Adoption', 'value': 'Adoption'},
                {'label': 'Return to Owner', 'value': 'Return to Owner'},
                {'label': 'Transer', 'value': 'Transfer'},
            ],\
            style={'display': 'flex', 'gap': '20px', 'justifyContent': 'center', 'padding': '10px'}
        ),
    ),
    ),
    html.Hr(),
    html.Div(className='buttonRow', style={'display': 'flex'}),
             #children=[html.Button(id='submit-cats', n_clicks=0, children='cats'),
                       #html.Button(id='submit-dogs', n_clicks=0, children='dogs')]),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable=False,
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################




    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(animal_selection):
## FIX ME Add code to filter interactive data table with MongoDB queries
    # Determine which button was clicked
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = None
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Default to all data
    query = {}

    if animal_selection == 'Cat':
        query = {"animal_type": "Cat"}
    elif animal_selection == 'Dog':
        query = {"animal_type": "Dog"}
    elif animal_selection == "Adoption":
        query = {"outcome_type": "Adoption"}
    elif animal_selection == "Return to Order":
        query = {"outcome_type": "Return to Owner"}
    elif animal_selection == "Transfer":
        query = {"outcome_type": "Transfer"}

    # Read from DB with query
    df = pd.DataFrame.from_records(db.read(query))

    # Drop MongoDB _id if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    return df.to_dict('records')
    

#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output("graph-id", "children"),
    Input("filter-type", "value"))
def update_graphs(animal_selection):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = None
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Default to all data
    query = {}

    if animal_selection == 'Cat':
        query = {"animal_type": "Cat"}
    elif animal_selection == 'Dog':
        query = {"animal_type": "Dog"}

    # Read from DB with query
    df = pd.DataFrame.from_records(db.read(query))

    # Drop MongoDB _id if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    chart_data = df.groupby('animal_type').size().reset_index(name='Count')
    chart_data.columns = ['Animal Type', 'Count']
  

    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    return [
        dcc.Graph(            
           figure = px.pie(chart_data, values='Count', names='Animal Type', title='Preferred Animals')
       )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row,13],dff.iloc[row,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://rapidrisk-noisestock-3000.codio.io/proxy/8050/
